In [ ]:
# 1. Load code/data
!git clone https://github.com/joms-hub/tagalog-fake-news-detection.git
import os
os.chdir('/kaggle/working/tagalog-fake-news-detection')

# 2. Install packages (if needed)
!pip install transformers datasets evaluate huggingface_hub accelerate torch

Cloning into 'tagalog-fake-news-detection'...
remote: Enumerating objects: 173, done.
remote: Counting objects: 100% (173/173), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 173 (delta 83), reused 126 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (173/173), 4.85 MiB | 19.77 MiB/s, done.
Resolving deltas: 100% (83/83), done.
  Using cached evaluate-0.4.5-py3-none-any.whl.metadata (9.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached evaluate-0.4.5-py3-none-any.whl (84 kB)
Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl (127.9 MB)
  Attempting uninstall: nvidia-cusolver-cu12
    Found existing installation: nvidia-cusolver-cu12 11.6.3.83
    Uninstalling nvidia-cusolver-cu12-11.6.3.83:
      Successfully uninstalled nvidia-cusolver-cu12-11.6.3.83


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!pip install nbstripout
!nbstripout '/content/drive/MyDrive/Colab Notebooks/electra-small-training.ipynb'

In [ ]:

# 3. Load preprocessed datasets
from datasets import load_from_disk
train_dataset = load_from_disk("/kaggle/working/tagalog-fake-news-detection/tokenized/ELECTRA-small_train")
val_dataset   = load_from_disk("/kaggle/working/tagalog-fake-news-detection/tokenized/ELECTRA-small_val")
test_dataset  = load_from_disk("/kaggle/working/tagalog-fake-news-detection/tokenized/ELECTRA-small_test")


In [ ]:
print(train_dataset, val_dataset, test_dataset)

Dataset({
    features: ['label', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2244
}) Dataset({
    features: ['label', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 481
}) Dataset({
    features: ['label', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 481
})


In [ ]:

# 4. Set up model
from transformers import ElectraForSequenceClassification
model_name = "google/electra-small-discriminator"
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=2)

2025-09-03 15:14:33.096058: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756912473.281137      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756912473.335189      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

In [ ]:

# 5. Training with early stopping
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from evaluate import load
import numpy as np

os.environ["WANDB_DISABLED"] = "true"  # Disable wandb logging

def compute_metrics(eval_pred):
    f1_metric = load("f1")
    acc_metric = load("accuracy")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    f1_result = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    acc_result = acc_metric.compute(predictions=predictions, references=labels)
    return {
        "f1": f1_result["f1"],
        "accuracy": acc_result["accuracy"]
    }

# Adjust batch sizes if you hit RAM limits (Kaggle RAM is 16GB)
training_args = TrainingArguments(
    output_dir='/kaggle/working/tagalog-fake-news-detection/results',
    num_train_epochs=10,
    per_device_train_batch_size=32,   # Reduce from 32 if you get OOM errors
    per_device_eval_batch_size=32,    # Same here
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir='/kaggle/working/tagalog-fake-news-detection/logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to=None
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,No log,0.474650,0.823092,0.825364
2,0.555100,0.344153,0.893846,0.893971
3,0.346500,0.347786,0.869548,0.871102
4,0.346500,0.257828,0.918904,0.918919
5,0.273300,0.260464,0.908373,0.908524
6,0.212500,0.221157,0.927217,0.927235
7,0.212500,0.217741,0.927159,0.927235
8,0.163600,0.205378,0.937573,0.937630
9,0.153500,0.194860,0.941777,0.941788
10,0.127400,0.195992,0.939694,0.939709


TrainOutput(global_step=710, training_loss=0.25971754738982294, metrics={'train_runtime': 290.8624, 'train_samples_per_second': 77.15, 'train_steps_per_second': 2.441, 'total_flos': 660176740270080.0, 'train_loss': 0.25971754738982294, 'epoch': 10.0})

In [ ]:
# 6. Logging and Saving Outputs
import json
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

label_names = ['Real', 'Fake']  # Update if your labels are different

predictions = trainer.predict(test_dataset)

output_dir = "/kaggle/working/tagalog-fake-news-detection/outputs"
os.makedirs(output_dir, exist_ok=True)

metrics = predictions.metrics
metrics_file = os.path.join(output_dir, 'electra-small_metrics.json')
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f"Saved metrics to {metrics_file}")

preds = np.argmax(predictions.predictions, axis=1)
cm = confusion_matrix(predictions.label_ids, preds)
cm_df = pd.DataFrame(cm, index=label_names, columns=[f'Predicted {n}' for n in label_names])
print("\nConfusion Matrix:")
print(cm_df)

cm_file = os.path.join(output_dir, 'electra-small_cm.csv')
cm_df.to_csv(cm_file)
print(f"Saved confusion matrix to {cm_file}")

results_df = pd.DataFrame({
    'true_label': predictions.label_ids,
    'predicted_label': preds
})
results_file = os.path.join(output_dir, 'electra-small_predictions.csv')
results_df.to_csv(results_file, index=False)
print(f"Saved raw predictions to {results_file}")

report = classification_report(predictions.label_ids, preds, target_names=label_names, output_dict=True)
report_file = os.path.join(output_dir, 'electra-small_classification_report.json')
with open(report_file, 'w') as f:
    json.dump(report, f, indent=2)
print("\nClassification Report:")
print(pd.DataFrame(report).transpose())
print(f"Saved classification report to {report_file}")


Saved metrics to /kaggle/working/tagalog-fake-news-detection/outputs/electra-small_metrics.json

Confusion Matrix:
      Predicted Real  Predicted Fake
Real             219              22
Fake              12             228
Saved confusion matrix to /kaggle/working/tagalog-fake-news-detection/outputs/electra-small_cm.csv
Saved raw predictions to /kaggle/working/tagalog-fake-news-detection/outputs/electra-small_predictions.csv

Classification Report:
              precision    recall  f1-score     support
Real           0.948052  0.908714  0.927966  241.000000
Fake           0.912000  0.950000  0.930612  240.000000
accuracy       0.929314  0.929314  0.929314    0.929314
macro avg      0.930026  0.929357  0.929289  481.000000
weighted avg   0.930063  0.929314  0.929286  481.000000
Saved classification report to /kaggle/working/tagalog-fake-news-detection/outputs/electra-small_classification_report.json


In [ ]:

# 7. Push trained model to HuggingFace
from huggingface_hub import login
login("")  # Replace with your token
trainer.push_to_hub("electra-small-fake-news-filipino")

Uploading...:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/jcunado/results/commit/b0e0130eb11df0da636d999e86617b91667c0231', commit_message='electra-small-fake-news-filipino', commit_description='', oid='b0e0130eb11df0da636d999e86617b91667c0231', pr_url=None, repo_url=RepoUrl('https://huggingface.co/jcunado/results', endpoint='https://huggingface.co', repo_type='model', repo_id='jcunado/results'), pr_revision=None, pr_num=None)

In [ ]:
%cd /kaggle/working/tagalog-fake-news-detection/
!git config --global user.name "joms-hub"
!git config --global user.email "21101932@usc.edu.ph"
!git remote set-url origin https://@github.com/joms-hub/tagalog-fake-news-detection.git

/kaggle/working/tagalog-fake-news-detection
rm 'notebooks/mobilebert_training.ipynb'
